In [1]:
from keybert import KeyBERT
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import pandas as pd
import torch
import os
from tqdm import tqdm, trange
from random import randint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

2023-09-05 15:11:16.187988: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-05 15:11:16.239827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag

In [3]:
vectorizer = TfidfVectorizer()
lemmatizer = WordNetLemmatizer()
stopword_list = stopwords.words('english')

In [4]:
tqdm.pandas()

In [5]:
def morphs(text, noun = True, verb = False, adjective = False, adverb = False):
    poses = pos_tag(text, tagset = 'universal')
    filters = []

    if noun:
        filters.append('NOUN')
    if verb:
        filters.append('VERB')
    if adjective:
        filters.append('ADJ')
    if adverb:
        filters.append('ADV')

    return [pos[0] for pos in poses if pos[1] in filters]

In [6]:
df1 = pd.read_csv('./datasets/original/roblox1.csv', index_col = 0, low_memory = False)
df2 = pd.read_csv('./datasets/original/roblox2.csv', index_col = 0, low_memory = False)
df3 = pd.read_csv('./datasets/original/roblox3.csv', index_col = 0, low_memory = False)
df4 = pd.read_csv('./datasets/original/roblox4.csv', index_col = 0, low_memory = False)
df5 = pd.read_csv('./datasets/original/roblox5.csv', index_col = 0, low_memory = False)
df6 = pd.read_csv('./datasets/original/zepeto.csv', index_col = 0, low_memory = False)

In [7]:
df = pd.concat([df1, df2, df3, df4, df5, df6]).reset_index(drop = True)

In [8]:
for df in [df1, df2, df3, df4, df5, df6]:
    df['keywords'] = df['content'].progress_apply(word_tokenize)                                                             # Tokenize
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if y.isalpha()])                                  # exclude if numeric
    df['keywords'] = df['keywords'].progress_apply(lambda x : [lemmatizer.lemmatize(y) for y in x])                           # lemmatize
    df['keywords'] = df['keywords'].progress_apply(lambda x : list(set(x)))                                                   # drop duplicate
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if len(y) >= 3 and len(y) <= 15])                 # 3 <= len(keyword) <= 15
    df['keywords'] = df['keywords'].progress_apply(morphs, noun = True, verb = True, adjective = False, adverb = False)       # select noun
    df['keywords'] = df['keywords'].progress_apply(lambda x : [y for y in x if y not in stopword_list])                       # stopwords
    df['keywords'] = df['keywords'].progress_apply(lambda x : ' '.join(x))

100%|██████████| 115407/115407 [00:00<00:00, 735271.53it/s]


In [9]:
df1.to_csv('./datasets/tfidf/roblox1.csv')
df2.to_csv('./datasets/tfidf/roblox2.csv')
df3.to_csv('./datasets/tfidf/roblox3.csv')
df4.to_csv('./datasets/tfidf/roblox4.csv')
df5.to_csv('./datasets/tfidf/roblox5.csv')
df6.to_csv('./datasets/tfidf/zepeto.csv')